In [1]:
# follwed from Andrej Karpathy Video "Let's build GPT"

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as Func

In [3]:
# hyperparameters:
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 8   # what is the maximum context length for predictions?
max_iters = 3000 # training iters
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

torch.manual_seed(2073)
model = None

In [4]:
with open('data/text.txt', encoding='utf-8') as fp:
    text = fp.read()

In [5]:
print(f"length of dataset in characters: {len(text)}\n")
print(f"\"{text[:60]}\" ")

length of dataset in characters: 1115393

"First Citizen:
Before we proceed any further, hear me speak." 


In [6]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(f"vocabulary: {''.join(vocab)}")
print(f"vocab size: {vocab_size}")

vocabulary: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


Simple Tokenization

In [7]:
# strategy to tokensize text -> model is character-level,
#  so we need to translate characters into integers.

# encode - string to integers (list)
def encode(input_string):
    # Define the vocabulary    
    # Create a dictionary to map each character to an integer
    char_to_index = {char: idx for idx, char in enumerate(vocab)}
    
    # Convert input string to an array of integers based on the character mapping
    encoded_array = list(map(lambda char: char_to_index[char], input_string))    
    return encoded_array


def decode(encoded_data):
    
    # mapping for integers back into string:
    index_to_char = {idx: char for idx, char in enumerate(vocab)}

    # convert the array of integers back to a string based on the reverse mapping:
    decoded_string = ''.join(map(lambda idx: index_to_char.get(idx, ''), encoded_data))
    return decoded_string

# Example usage of encode and decode
input_str = "hii there"
encoded_output = encode(input_str)
print("Encoded:", encoded_output)

decoded_output = decode(encoded_output)
print("Decoded:", decoded_output)

Encoded: [46, 47, 47, 1, 58, 46, 43, 56, 43]
Decoded: hii there


In [8]:
# Time to encode dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
#print(data[:1000])

torch.Size([1115393]) torch.int64


Train / Test split

In [9]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Training

In [10]:
# block : random sampled 'chunks' of dataset
# block_size = 8
train_data[:block_size+1]
# +1 because for a chunk of size 8, need to see what comes next?

x = train_data[:block_size]
y = train_data[1:block_size+1]

# y are the targets for each position in the input

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

# we train on full range of context from 1 to block_size, because
# this makes transformer 'used to' seeing contexts in that range since it
# improves sampling during inference generality

# after block size we need ot truncate, since transformer can never recieve more than block_size inputs.


when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


Batching

In [11]:
# batch dimension: we train on multiple different chunks of text,
# each with size block_size, to leverage gpu acceleration

#batch_size = 4
#block_size = 8

device_model = None

def get_batch(split):
    # gen a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    device_model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = device_model(X, Y)
            losses[k] = loss.item()

        out[split] = losses.mean()
    
    device_model.train()
    return out

xb, yb = get_batch('train')

print(f"inputs: {xb.shape}, \n{xb}\n\ntargets: {yb.shape}\n{yb}")

print('------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is: {target}")



inputs: torch.Size([32, 8]), 
tensor([[56, 39, 58, 43,  1, 51, 39, 52],
        [57, 50, 47, 54,  1, 39, 58,  1],
        [53, 52, 58, 39, 45, 59, 43,  6],
        [ 1, 14, 43, 56, 49, 43, 50, 43],
        [42,  1, 53, 52, 43,  7,  7, 61],
        [57,  6,  1, 61, 46, 53,  0, 20],
        [51, 43, 53,  8,  0,  0, 22, 33],
        [61, 47, 44, 43,  6,  1, 57, 47],
        [54, 53, 61, 43, 56,  1, 57, 58],
        [ 1, 53, 59, 56,  1, 54, 39, 50],
        [39, 51, 43,  1, 51, 53, 56, 43],
        [ 1, 44, 50, 53, 61,  1, 61, 47],
        [42,  1, 45, 43, 52, 58, 50, 43],
        [43,  6,  1, 39, 52, 42,  1, 50],
        [58, 46, 63, 57, 43, 50, 44, 12],
        [ 1, 51, 63,  1, 40, 59, 56, 58],
        [57, 61, 47, 51, 11,  1, 39, 50],
        [ 1, 56, 43, 54, 56, 53, 53, 44],
        [46,  1, 57, 59, 41, 49,  5, 42],
        [50, 50,  7, 51, 63,  7, 42, 39],
        [ 1, 41, 39, 52,  1, 39, 47, 42],
        [42, 43, 57, 58, 56, 59, 41, 58],
        [56,  6,  0, 32, 53,  1, 40, 43],
    

In [12]:
print(xb)

tensor([[56, 39, 58, 43,  1, 51, 39, 52],
        [57, 50, 47, 54,  1, 39, 58,  1],
        [53, 52, 58, 39, 45, 59, 43,  6],
        [ 1, 14, 43, 56, 49, 43, 50, 43],
        [42,  1, 53, 52, 43,  7,  7, 61],
        [57,  6,  1, 61, 46, 53,  0, 20],
        [51, 43, 53,  8,  0,  0, 22, 33],
        [61, 47, 44, 43,  6,  1, 57, 47],
        [54, 53, 61, 43, 56,  1, 57, 58],
        [ 1, 53, 59, 56,  1, 54, 39, 50],
        [39, 51, 43,  1, 51, 53, 56, 43],
        [ 1, 44, 50, 53, 61,  1, 61, 47],
        [42,  1, 45, 43, 52, 58, 50, 43],
        [43,  6,  1, 39, 52, 42,  1, 50],
        [58, 46, 63, 57, 43, 50, 44, 12],
        [ 1, 51, 63,  1, 40, 59, 56, 58],
        [57, 61, 47, 51, 11,  1, 39, 50],
        [ 1, 56, 43, 54, 56, 53, 53, 44],
        [46,  1, 57, 59, 41, 49,  5, 42],
        [50, 50,  7, 51, 63,  7, 42, 39],
        [ 1, 41, 39, 52,  1, 39, 47, 42],
        [42, 43, 57, 58, 56, 59, 41, 58],
        [56,  6,  0, 32, 53,  1, 40, 43],
        [51, 43,  1, 43, 52, 53, 5

Building our Neural Network

In [13]:
# bigram language model is the simplest model for nlp.
# bigram predicts next word solely based on the preceeding word. (n-gram where n = 2)
# P(W)=P(w1​)⋅P(w2​∣w1​)⋅P(w3​∣w2​)⋅…⋅P(wn​∣wn−1​)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits of the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
            # idx and targets are both (B,T) tensor of integers
            logits = self.token_embedding_table(idx) # (B,T,C)
            # B, T, C: batch, time, channel

            if targets is None:
                loss = None
            else:
                # loss fn:
                # measures quality of logits w.r.t. the targets 
                # we have identity of next char, how well are we predicting next char based on logits?
                
                #pytorch expects different format for cross_entropy (B,C,T)
                # we need to reshape logits:
                B, T, C = logits.shape

                logits = logits.view(B*T, C)
                targets = targets.view(B*T)
                
                loss = Func.cross_entropy(logits, targets)



            return logits, loss


    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context

        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)

             # focus only on the last time step:
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmas to get probabilities:
            probs = Func.softmax(logits, dim=-1) # (B, C)

            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    



model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
# expecting loss to be (-1) * ln(1/65) = ~ 4.17

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# load model to device: 
device_model = model.to(device)

# create a pytorch optimizer:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)


# Training loop:
for steps in range(max_iters):

    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# generate from the model :
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(device_model.generate(context, max_new_tokens=500)[0].tolist()))

: 

In [ ]:
# in the bigram language model, the tokens are not talking to eachother, so of course its going to be jibberish.
# though its more coherent after some training, its still very nonsensical.
# bigram is simple but also not very powerful.

# current impl only uses the last character to predict what comes next.


# how to get tokens to talk to eachother?

: 

### The mathematical trick in self-attention

In [ ]:
# considre the following toy example:

B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape


: 

: 